# Code-along Movie Recommendation System

Source: [Building a Movie Recommendation system | K-Nearest Neighbors | Machine Learning](https://www.youtube.com/watch?v=4Ws0oPH350U&ab_channel=WennovationAcademy)

In [1]:
import pandas as pd

In [11]:
movies = "../assets/movies.csv"
ratings = "../assets/ratings.csv"

df_movies = pd.read_csv(movies, usecols = ["movieId", "title"], dtype = dict(movieId = "int32", title = "str"))
df_ratings = pd.read_csv(ratings, usecols = ["userId", "movieId", "rating"], dtype = dict(userId = "int32", movieId = "int32", rating = "float32"))

In [12]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [13]:
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


---
### Looking at dataset values

In [3]:
df_movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [5]:
df_ratings.describe()

,userId,movieId,rating
count,100836.000000,100836.000000,100836.000000
mean,326.127564,19435.295718,3.501557
std,182.618491,35530.987199,1.042529
min,1.000000,1.000000,0.500000
25%,177.000000,1199.000000,3.000000
50%,325.000000,2991.000000,3.500000
75%,477.000000,8122.000000,4.000000
max,610.000000,193609.000000,5.000000


In [14]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int32 
 1   title    9742 non-null   object
dtypes: int32(1), object(1)
memory usage: 114.3+ KB


In [10]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int32  
 1   movieId  100836 non-null  int32  
 2   rating   100836 non-null  float32
dtypes: float32(1), int32(2)
memory usage: 1.2 MB


---
### Sparse Matrix

In [19]:
# Sparse Matrix
#
#           Users
# Movies -> [4, 4, 5] A
#        -> [3, 3, 4] B      Cos(A, B) -> 0.95 (some similarity value)
#        -> [3, 2, 1]

movies_users = df_ratings.pivot(index = "movieId", columns = "userId", values = "rating").fillna(0)

movies_users

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
from scipy.sparse import csr_matrix

mat_movies_users = csr_matrix(movies_users.values)

mat_movies_users

<9724x610 sparse matrix of type '<class 'numpy.float32'>'
	with 100836 stored elements in Compressed Sparse Row format>

---
### Create Model

In [23]:
from sklearn.neighbors import NearestNeighbors


# Euclidean Distance
# Manhattan Distance
# Minkowski Distance

# Cosine Simularity is used here
model_KNN = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors = 20)

model_KNN.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

---
### Recommender Function

In [38]:
from fuzzywuzzy import process

# Recommender (movie_name) -> list of recommended movies

def recommender(movie_name, data, model, n_recommendations):
    model.fit(data)
    idx = process.extractOne(movie_name, df_movies["title"])[2] # format: ('Toy Story (1995)', 90, 0) -> index [2] gives position in dataframe
    print(f"Movie Selected: {df_movies['title'][idx]}, Index: {idx}")
    print("Searching for recommendations...")

    distances, indices = model.kneighbors(data[idx])
    for i in indices:
        print(df_movies["title"][i].where(i != idx)) # where skips recommending same movie

recommender("toy story", mat_movies_users, model_KNN, 20)

Movie Selected: Toy Story (1995), Index: 0
Searching for recommendations...
0                                                     NaN
2353                                 'night Mother (1986)
418                                  Jurassic Park (1993)
615                  Independence Day (a.k.a. ID4) (1996)
224             Star Wars: Episode IV - A New Hope (1977)
314                                   Forrest Gump (1994)
322                                 Lion King, The (1994)
910     Once Upon a Time in the West (C'era una volta ...
546                            Mission: Impossible (1996)
963                                           Diva (1981)
968                           Arsenic and Old Lace (1944)
3189            Rififi (Du rififi chez les hommes) (1955)
506                                        Aladdin (1992)
123                                      Apollo 13 (1995)
257                                   Pulp Fiction (1994)
897                 Cheech and Chong's Up in Smoke (19